In [1]:
import re
import os
w_nb = 20 # number of words before linebreaks in output

In [9]:
# import the lexicon
with open('tsikchen.txt', 'r', -1, 'utf-8-sig') as f:
    lexicon = [line.strip() for line in f.readlines()]
# add all the particles
lexicon.extend(['གི', 'ཀྱི', 'གྱི', 'ཡི', 'གིས', 'ཀྱིས', 'གྱིས', 'ཡིས', 'སུ', 'ཏུ', 'དུ', 'རུ', 'སྟེ', 'ཏེ', 'དེ', 'ཀྱང', 'ཡང', 'འང', 'གམ', 'ངམ', 'དམ', 'ནམ', 'བམ', 'མམ', 'འམ', 'རམ', 'ལམ', 'སམ', 'ཏམ', 'གོ', 'ངོ', 'དོ', 'ནོ', 'མོ', 'འོ', 'རོ', 'ལོ', 'སོ', 'ཏོ', 'ཅིང', 'ཅེས', 'ཅེའོ', 'ཅེ་ན', 'ཅིག', 'ཞིང', 'ཞེས', 'ཞེའོ', 'ཞེ་ན', 'ཞིག', 'ཤིང', 'ཤེའོ', 'ཤེ་ན', 'ཤིག', 'ལ', 'ན', 'ནས', 'ལས', 'ནི', 'དང', 'གང', 'ཅི', 'ཇི', 'གིན', 'གྱིན', 'ཀྱིན', 'ཡིན', 'པ', 'བ', 'པོ', 'བོ'])
# add all Monlam verbs
with open('monlam1_verbs.txt', 'r', -1, 'utf-8-sig') as f:
    monlam_verbs = [line.strip() for line in f.readlines()]
for entry in monlam_verbs:
    verb = entry.split(' | ')[0]
    if verb not in lexicon:
        lexicon.append(verb)

# import the Monlam POS tags
with open('./monlam1_pos.txt', 'r', -1, 'utf-8-sig') as f: # Monlam
    monlam = [line.strip() for line in f.readlines()]
monlam_pos = {}
for line in monlam:
    parts = line.split('—') # Monlam
    monlam_pos[parts[0]] = parts[1] 

# import the Hill POS tags
with open('hill_pos.txt', 'r', -1, 'utf-8-sig') as f: # Hill
    hill = [line.strip() for line in f.readlines()]
hill_pos = {}
for line in hill:
    parts = line.split('***') # Hill
    hill_pos[parts[0]] = parts[1]

ཧྲོད


new_tsikchen = []
no_pos = []
for word in lexicon:
    word = word[:-1]
    if word in monlam_pos.keys() or word in hill_pos.keys():
        new_tsikchen.append(word)
    else:
        no_pos.append(word)

with open('tsikchen.txt', 'w', -1, 'utf-8-sig') as f:
    f.write('\n'.join(new_tsikchen))

In [5]:
def isWord(syls):
    maybe = '་'.join(syls)
    final = False
    if maybe in lexicon:
        final = True
    elif re.sub(merged_part, '', maybe) in lexicon:
        final = True
    return final

def process(list1, list2, num):
    word = '་'.join(list1[:num])
    if word not in lexicon:
        maybe = re.split(merged_part, word)
        list2.append(maybe[0])
        list2.append(maybe[1]+'་')
        del list1[:num]
    else:
        list2.append(word+"་")
        del list1[:num]

In [12]:
for file in os.listdir('./IN/'):
    if file.endswith(".txt"):
    #todo - replace \n by \s try: with open('drugs') as temp_file: \n drugs = [line.rstrip('\n') for line in temp_file]
        try:
            with open('./IN/' + file, 'r', -1, 'utf-8-sig') as f:
                current_file = f.read().replace('\n', '').replace('\r\n', '')

        except:
            print("Save all IN files as UTF-8 and try again.")
            input()
    else:
        print("\nSave all IN files as text files and try again.")
        input()
    
    ######################
    # Segmentation process
    merged_part = r'(ར|ས|འི|འམ|འང)$'
    
    syls = re.sub(r"([།|༎|༏|༐|༑|༔|\s]+)", "་\g<1>་", current_file)
    syls = re.split(r"་+", syls)
    
    non_words = []
    words = []
    while len(syls) > 0:
        if   isWord(syls[:4]): process(syls, words, 4)
        elif isWord(syls[:3]): process(syls, words, 3)
        elif isWord(syls[:2]): process(syls, words, 2)
        elif isWord(syls[:1]): process(syls, words, 1)
        else:
            words.append('་'.join(syls[:1])+"་*")
            non_words.append('་'.join(syls[:1])+"་")
            del syls[:1]
    #
    ######################
    pos_tagged = []
    for word in words:
        wor = word
        if word.endswith('་'):
            wor = word[:-1]
        if wor in monlam_pos.keys() and wor in hill_pos.keys():
            pos_tagged.append(word+'|'+hill_pos[wor]+'#'+monlam_pos[wor])
        elif wor in monlam_pos.keys() and wor not in hill_pos.keys():
            pos_tagged.append(word+'|'+monlam_pos[wor])
        elif wor not in monlam_pos.keys() and wor in hill_pos.keys():
            pos_tagged.append(word+'|'+hill_pos[wor])
        else:
            pos_tagged.append(word)
    
    ######################
    # count percentage of POS tagged words
    
    pos = 0
    no_pos = 0
    for word in pos_tagged:
        if '|' in word:
            pos = pos+1
        else:
            no_pos = pos+1
    print(pos, 'words out of', len(pos_tagged), 'have a POS.', '\n', pos*100/len(pos_tagged), '% of the words have tags.')
    
    #
    ######################
    
    ######################
    # flag particles
    
    
    #
    ######################
    
    ######################
    # flag verbs
    
    
    #
    ######################
    

    # add linebreaks after 400 words
    for i in range(w_nb-1, len(pos_tagged), w_nb):
        words[i] += '\n'
    
    # write output
    with open('./OUT/' + 'seg_' + file, 'w', -1, 'utf-8-sig') as f:
        f.write(' '.join(words))
    
    with open('nonwords_' + file, 'w', -1, 'utf-8-sig') as f:
        f.write('\n'.join(non_words))

881 words out of 1094 have a POS. 
 80.53016453382084 % of the words have tags.
17 words out of 19 have a POS. 
 89.47368421052632 % of the words have tags.
